In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing all the dependencies

In [ ]:
# !pip install tensorflow_text

In [ ]:
# !pip install sentencepiece 
# !pip install tensorflow_addons
# !pip install gin

In [ ]:
import tensorflow as tf
!git clone --depth 1 -b v2.3.0 https://github.com/tensorflow/models.git

import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import sys
sys.path.append('models')
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
from helper_functions import *

# sys.path.append('models')
from official.nlp.data import classifier_data_lib
from official.nlp.bert import tokenization
# from official.nlp import optimization

import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# DATA_URL = "https://archive.org/download/fine-tune-bert-tensorflow-train.csv/train.csv.zip"
# df = pd.read_csv(DATA_URL, compression="zip", low_memory=False)

DATA_URL = "https://raw.githubusercontent.com/gkv856/KaggleData/main/train.csv"
TEST_URL = "https://raw.githubusercontent.com/gkv856/KaggleData/main/test.csv"

train = pd.read_csv(DATA_URL, low_memory=False)
test = pd.read_csv(TEST_URL, low_memory=False)

print(tf.version.VERSION)

# EDA - exploratory data analysis

In [ ]:
train.shape, test.shape, test.shape[0]/train.shape[0]

In [ ]:
print("There are '{}' rows and '{}' columns in training data".format(train.shape[0],train.shape[1]))
print("There are '{}' rows and '{}' columns in test data".format(test.shape[0],test.shape[1]))

In [ ]:
train.info()

In [ ]:
null_counts = pd.DataFrame({"Num_Null": train.isnull().sum()})
null_counts["Pct_Null"] = null_counts["Num_Null"] / train.count() * 100
null_counts

In [ ]:
keywords_vc = pd.DataFrame({"Count": train["keyword"].value_counts()})
sns.barplot(y=keywords_vc[0:30].index, x=keywords_vc[0:30]["Count"], orient='h')
plt.title("Top 30 Keywords")
plt.show()
plt.figure(figsize=(40, 40))

In [ ]:

# total unique keywords
len(train["keyword"].value_counts())

In [ ]:
disaster_keywords = train.loc[train["target"] == 1]["keyword"].value_counts()
nondisaster_keywords = train.loc[train["target"] == 0]["keyword"].value_counts()

fig, ax = plt.subplots(1,2, figsize=(20,8))
sns.barplot(y=disaster_keywords[0:30].index, x=disaster_keywords[0:30], orient='h', ax=ax[0], palette="Reds_d")
sns.barplot(y=nondisaster_keywords[0:30].index, x=nondisaster_keywords[0:30], orient='h', ax=ax[1], palette="Blues_d")
ax[0].set_title("Top 30 Keywords - Disaster Tweets")
ax[0].set_xlabel("Keyword Frequency")
ax[1].set_title("Top 30 Keywords - Non-Disaster Tweets")
ax[1].set_xlabel("Keyword Frequency")
plt.tight_layout()
plt.show()

In [ ]:
mask = (train["keyword"].fillna("").str.contains("armageddon")) & (train["target"] == 0)
armageddon_tweets = train[mask]
print("An example tweet from 10th row:\n", armageddon_tweets.iloc[10, 3])
armageddon_tweets.head()

In [ ]:
def keyword_disaster_probabilities(x):
    tweets_w_keyword = np.sum(train["keyword"].fillna("").str.contains(x))
    tweets_w_keyword_disaster = np.sum(train["keyword"].fillna("").str.contains(x) & train["target"] == 1)
    return tweets_w_keyword_disaster / tweets_w_keyword

keywords_vc["Disaster_Probability"] = keywords_vc.index.map(keyword_disaster_probabilities)
keywords_vc.head()

In [ ]:

# sorting the diaster probabilty
k_vc = keywords_vc.sort_values(by="Disaster_Probability", ascending=False)
k_vc.head(10)

In [ ]:
k_vc.tail(10)

In [ ]:
locations_vc = train["location"].value_counts()
sns.barplot(y=locations_vc[0:30].index, x=locations_vc[0:30], orient='h')
plt.title("Top 30 Locations")
plt.show()

In [ ]:
disaster_locations = train.loc[train["target"] == 1]["location"].value_counts()
nondisaster_locations = train.loc[train["target"] == 0]["location"].value_counts()

fig, ax = plt.subplots(1,2, figsize=(20,8))
sns.barplot(y=disaster_locations[0:30].index, x=disaster_locations[0:30], orient='h', ax=ax[0], palette="Reds_d")
sns.barplot(y=nondisaster_locations[0:30].index, x=nondisaster_locations[0:30], orient='h', ax=ax[1], palette="Blues_d")
ax[0].set_title("Top 30 Locations - Disaster Tweets")
ax[0].set_xlabel("Keyword Frequency")
ax[1].set_title("Top 30 Locations - Non-Disaster Tweets")
ax[1].set_xlabel("Keyword Frequency")
plt.tight_layout()
plt.show()

In [ ]:
train["tweet_length"] = train["text"].apply(len)
sns.distplot(train["tweet_length"])
plt.title("Histogram of Tweet Length")
plt.xlabel("Number of Characters")
plt.ylabel("Density")
plt.show()

In [ ]:
# min and max tweet length
min(train["tweet_length"]), max(train["tweet_length"])

In [ ]:
# histogram for each category of tweet
g = sns.FacetGrid(train, col="target", height=5)
g = g.map(sns.histplot, "tweet_length")
plt.suptitle("Distribution Tweet Length")
plt.show()

In [ ]:
def count_words(x):
    return len(x.split())

train["num_words"] = train["text"].apply(count_words)
sns.distplot(train["num_words"], bins=10)
plt.title("Histogram of Number of Words per Tweet")
plt.xlabel("Number of Words")
plt.ylabel("Density")
plt.show()

In [ ]:
g = sns.FacetGrid(train, col="target", height=5)
g = g.map(sns.distplot, "num_words")
plt.suptitle("Distribution Number of Words")
plt.show()

In [ ]:
def avg_word_length(x):
    return np.sum([len(w) for w in x.split()]) / len(x.split())

train["avg_word_length"] = train["text"].apply(avg_word_length)
sns.distplot(train["avg_word_length"])
plt.title("Histogram of Average Word Length")
plt.xlabel("Average Word Length")
plt.ylabel("Density")
plt.show()

In [ ]:
train

In [ ]:
# our data is imbalanced and we will assume that in the real world/test data, 
# this will continued to be the case 
train["target"].plot(kind="hist")

In [ ]:
# later you can use the whole data for traiing and test
train_df, valid_df = train_test_split(train, 
                                       random_state=42, 
                                      #  train_size=0.9, 
                                       test_size=.25,
                                       stratify=train["target"].values)

# use the below structure for testing if the data is huge
# train_df, remaining = train_test_split(train, 
#                                        random_state=42, 
#                                        train_size=0.0095, 
#                                        stratify=train["target"].values)

# valid_df, _ = train_test_split(remaining, 
#                               random_state=42, 
#                               train_size=0.0095, 
#                               stratify=remaining["target"].values)
len(train_df), len(valid_df)

In [ ]:
# creating datasets
with tf.device('/cpu:0'):
  train_data = tf.data.Dataset.from_tensor_slices((train_df["text"].values,
                                                   train_df["target"].values))
  
  test_data = tf.data.Dataset.from_tensor_slices((valid_df["text"].values,
                                                   valid_df["target"].values))
train_data, test_data

In [ ]:
for text, label in train_data.take(1):
  print(text)
  print(label)

In [ ]:
"""
Each line of the dataset is composed of the review text and its label
- Data preprocessing consists of transforming text to BERT input features:
input_word_ids, input_mask, segment_ids
- In the process, tokenizing the text is done with the provided BERT model tokenizer
"""

# Label categories, right now our data has these categories
label_list = [0, 1]

# maximum length of (token) input sequences, or the words in a question
# to save speed we should reset this
max_seq_length = 128

train_batch_size = 32

In [ ]:
# Get BERT layer and tokenizer:
# More details here: https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2
MODEL_URL = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"
# MODEL_URL = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2"
bert_layer = hub.KerasLayer(MODEL_URL, trainable=True)


vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

# Task 6: Tokenize and Preprocess Text for BERT

In [ ]:
# This provides a function to convert row to input features and label
from bs4 import BeautifulSoup
import re
import string

abbreviations = {
    "$" : " dollar ",
    "€" : " euro ",
    "4ao" : "for adults only",
    "a.m" : "before midday",
    "a3" : "anytime anywhere anyplace",
    "aamof" : "as a matter of fact",
    "acct" : "account",
    "adih" : "another day in hell",
    "afaic" : "as far as i am concerned",
    "afaict" : "as far as i can tell",
    "afaik" : "as far as i know",
    "afair" : "as far as i remember",
    "afk" : "away from keyboard",
    "app" : "application",
    "approx" : "approximately",
    "apps" : "applications",
    "asap" : "as soon as possible",
    "asl" : "age, sex, location",
    "atk" : "at the keyboard",
    "ave." : "avenue",
    "aymm" : "are you my mother",
    "ayor" : "at your own risk", 
    "b&b" : "bed and breakfast",
    "b+b" : "bed and breakfast",
    "b.c" : "before christ",
    "b2b" : "business to business",
    "b2c" : "business to customer",
    "b4" : "before",
    "b4n" : "bye for now",
    "b@u" : "back at you",
    "bae" : "before anyone else",
    "bak" : "back at keyboard",
    "bbbg" : "bye bye be good",
    "bbc" : "british broadcasting corporation",
    "bbias" : "be back in a second",
    "bbl" : "be back later",
    "bbs" : "be back soon",
    "be4" : "before",
    "bfn" : "bye for now",
    "blvd" : "boulevard",
    "bout" : "about",
    "brb" : "be right back",
    "bros" : "brothers",
    "brt" : "be right there",
    "bsaaw" : "big smile and a wink",
    "btw" : "by the way",
    "bwl" : "bursting with laughter",
    "c/o" : "care of",
    "cet" : "central european time",
    "cf" : "compare",
    "cia" : "central intelligence agency",
    "csl" : "can not stop laughing",
    "cu" : "see you",
    "cul8r" : "see you later",
    "cv" : "curriculum vitae",
    "cwot" : "complete waste of time",
    "cya" : "see you",
    "cyt" : "see you tomorrow",
    "dae" : "does anyone else",
    "dbmib" : "do not bother me i am busy",
    "diy" : "do it yourself",
    "dm" : "direct message",
    "dwh" : "during work hours",
    "e123" : "easy as one two three",
    "eet" : "eastern european time",
    "eg" : "example",
    "embm" : "early morning business meeting",
    "encl" : "enclosed",
    "encl." : "enclosed",
    "etc" : "and so on",
    "faq" : "frequently asked questions",
    "fawc" : "for anyone who cares",
    "fb" : "facebook",
    "fc" : "fingers crossed",
    "fig" : "figure",
    "fimh" : "forever in my heart", 
    "ft." : "feet",
    "ft" : "featuring",
    "ftl" : "for the loss",
    "ftw" : "for the win",
    "fwiw" : "for what it is worth",
    "fyi" : "for your information",
    "g9" : "genius",
    "gahoy" : "get a hold of yourself",
    "gal" : "get a life",
    "gcse" : "general certificate of secondary education",
    "gfn" : "gone for now",
    "gg" : "good game",
    "gl" : "good luck",
    "glhf" : "good luck have fun",
    "gmt" : "greenwich mean time",
    "gmta" : "great minds think alike",
    "gn" : "good night",
    "g.o.a.t" : "greatest of all time",
    "goat" : "greatest of all time",
    "goi" : "get over it",
    "gps" : "global positioning system",
    "gr8" : "great",
    "gratz" : "congratulations",
    "gyal" : "girl",
    "h&c" : "hot and cold",
    "hp" : "horsepower",
    "hr" : "hour",
    "hrh" : "his royal highness",
    "ht" : "height",
    "ibrb" : "i will be right back",
    "ic" : "i see",
    "icq" : "i seek you",
    "icymi" : "in case you missed it",
    "idc" : "i do not care",
    "idgadf" : "i do not give a damn fuck",
    "idgaf" : "i do not give a fuck",
    "idk" : "i do not know",
    "ie" : "that is",
    "i.e" : "that is",
    "ifyp" : "i feel your pain",
    "IG" : "instagram",
    "iirc" : "if i remember correctly",
    "ilu" : "i love you",
    "ily" : "i love you",
    "imho" : "in my humble opinion",
    "imo" : "in my opinion",
    "imu" : "i miss you",
    "iow" : "in other words",
    "irl" : "in real life",
    "j4f" : "just for fun",
    "jic" : "just in case",
    "jk" : "just kidding",
    "jsyk" : "just so you know",
    "l8r" : "later",
    "lb" : "pound",
    "lbs" : "pounds",
    "ldr" : "long distance relationship",
    "lmao" : "laugh my ass off",
    "lmfao" : "laugh my fucking ass off",
    "lol" : "laughing out loud",
    "ltd" : "limited",
    "ltns" : "long time no see",
    "m8" : "mate",
    "mf" : "motherfucker",
    "mfs" : "motherfuckers",
    "mfw" : "my face when",
    "mofo" : "motherfucker",
    "mph" : "miles per hour",
    "mr" : "mister",
    "mrw" : "my reaction when",
    "ms" : "miss",
    "mte" : "my thoughts exactly",
    "nagi" : "not a good idea",
    "nbc" : "national broadcasting company",
    "nbd" : "not big deal",
    "nfs" : "not for sale",
    "ngl" : "not going to lie",
    "nhs" : "national health service",
    "nrn" : "no reply necessary",
    "nsfl" : "not safe for life",
    "nsfw" : "not safe for work",
    "nth" : "nice to have",
    "nvr" : "never",
    "nyc" : "new york city",
    "oc" : "original content",
    "og" : "original",
    "ohp" : "overhead projector",
    "oic" : "oh i see",
    "omdb" : "over my dead body",
    "omg" : "oh my god",
    "omw" : "on my way",
    "p.a" : "per annum",
    "p.m" : "after midday",
    "pm" : "prime minister",
    "poc" : "people of color",
    "pov" : "point of view",
    "pp" : "pages",
    "ppl" : "people",
    "prw" : "parents are watching",
    "ps" : "postscript",
    "pt" : "point",
    "ptb" : "please text back",
    "pto" : "please turn over",
    "qpsa" : "what happens", #"que pasa",
    "ratchet" : "rude",
    "rbtl" : "read between the lines",
    "rlrt" : "real life retweet", 
    "rofl" : "rolling on the floor laughing",
    "roflol" : "rolling on the floor laughing out loud",
    "rotflmao" : "rolling on the floor laughing my ass off",
    "rt" : "retweet",
    "ruok" : "are you ok",
    "sfw" : "safe for work",
    "sk8" : "skate",
    "smh" : "shake my head",
    "sq" : "square",
    "srsly" : "seriously", 
    "ssdd" : "same stuff different day",
    "tbh" : "to be honest",
    "tbs" : "tablespooful",
    "tbsp" : "tablespooful",
    "tfw" : "that feeling when",
    "thks" : "thank you",
    "tho" : "though",
    "thx" : "thank you",
    "tia" : "thanks in advance",
    "til" : "today i learned",
    "tl;dr" : "too long i did not read",
    "tldr" : "too long i did not read",
    "tmb" : "tweet me back",
    "tntl" : "trying not to laugh",
    "ttyl" : "talk to you later",
    "u" : "you",
    "u2" : "you too",
    "u4e" : "yours for ever",
    "utc" : "coordinated universal time",
    "w/" : "with",
    "w/o" : "without",
    "w8" : "wait",
    "wassup" : "what is up",
    "wb" : "welcome back",
    "wtf" : "what the fuck",
    "wtg" : "way to go",
    "wtpa" : "where the party at",
    "wuf" : "where are you from",
    "wuzup" : "what is up",
    "wywh" : "wish you were here",
    "yd" : "yard",
    "ygtr" : "you got that right",
    "ynk" : "you never know",
    "zzz" : "sleeping bored and tired"
}
# Change an abbreviation by its true meaning
def word_abbrev(word):
    return abbreviations[word.lower()] if word.lower() in abbreviations.keys() else word

# Replace all abbreviations
def replace_abbrev(text):
    string = ""
    for word in text.split():
        string += word_abbrev(word) + " "        
    return string

# Remove all emojis, replace by EMOJI
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'EMOJI', text)

# Replace some others smileys with SADFACE
def transcription_sad(text):
    eyes = "[8:=;]"
    nose = "['`\-]"
    smiley = re.compile(r'[8:=;][\'\-]?[(\\/]')
    return smiley.sub(r'SADFACE', text)

# Replace some smileys with SMILE
def transcription_smile(text):
    eyes = "[8:=;]"
    nose = "['`\-]"
    smiley = re.compile(r'[8:=;][\'\-]?[)dDp]')
    #smiley = re.compile(r'#{eyes}#{nose}[)d]+|[)d]+#{nose}#{eyes}/i')
    return smiley.sub(r'SMILE', text)

# Replace <3 with HEART
def transcription_heart(text):
    heart = re.compile(r'<3')
    return heart.sub(r'HEART', text)

# Factorize elongated words, add ELONG
def remove_elongated_words(text):
    rep = re.compile(r'\b(\S*?)([a-z])\2{2,}\b')
    return rep.sub(r'\1\2 ELONG', text)

# Factorize repeated punctuation, add REPEAT
def remove_repeat_punct(text):
    rep = re.compile(r'([!?.]){2,}')
    return rep.sub(r'\1 REPEAT', text)


# Remove all punctuations
def remove_all_punct(text):
    table = str.maketrans('','',string.punctuation)
    return text.translate(table)

# Remove punctuations
def remove_punct(text):
    punctuations = '@#!?+&*[]-%.:/();$=><|{}^' + "'`" 
    for p in punctuations:
        text = text.replace(p, f' {p} ')

    text = text.replace('...', ' ... ')
    if '...' not in text:
        text = text.replace('..', ' ... ')   
    return text

# Remove all english stopwords
def remove_stopwords(text):
    text = ' '.join([word for word in text.split() if word not in (stopwords)])
    return text

def get_clean_text(text_str):
  text_str = BeautifulSoup(text_str, "lxml").get_text()
  text_str = re.sub(r"@[A-Za-z0-9]+", " ", text_str)

  # replacing URLs with just 'URL' text
  text_str = re.sub(r"https?://[[A-Za-z0-9./]+", "URL", text_str)
  
  # remove html beacons
  text_str = re.sub(r"<.*?>", " ", text_str)

  # replacing @someuser to 'USER' text
  text_str = re.sub(r"@\S+", "USER", text_str)

  # replacing numbers to 'NUMBER' text
  text_str = re.sub(r"[-+]?[.\d]*[\d]+[:,.\d]*", "NUMBER", text_str)

  # correcting the abbreviated words to real meaning
  text_str = replace_abbrev(text_str)

  # Remove emojis / smileys
  text_str = remove_emoji(text_str)
  text_str = transcription_sad(text_str)
  text_str = transcription_smile(text_str)
  text_str = transcription_heart(text_str)
  
  # # Remove repeated puntuations / words
  text_str = remove_elongated_words(text_str)
  text_str = remove_repeat_punct(text_str)

  # text_str = remove_all_punct(text_str)
  # text_str = remove_punct(text_str)
  # text_str = remove_stopwords(text_str)


  text_str = re.sub(r"[^a-zA-Z.!?']", " ", text_str)
  text_str = re.sub(r" +", " ", text_str)
  
  


  return text_str

def to_feature(text, label, label_list=label_list, max_seq_length=max_seq_length, tokenizer=tokenizer):
  # guid is unique id for each example, we dont need textb as per our usecase
  clean_text = get_clean_text(text.numpy())
  # clean_text = text.numpy()

  example = classifier_data_lib.InputExample(guid=None, text_a=clean_text, label=label.numpy())

  feature = classifier_data_lib.convert_single_example(0, 
                                                       example, 
                                                       label_list, 
                                                       max_seq_length, 
                                                       tokenizer)
  
  return (feature.input_ids, feature.input_mask, feature.segment_ids, feature.label_id)

In [ ]:
# testing the feature function
# ct= get_clean_text("@ENews Ben Affleck......I know there's a wife/kids and other girls but I can't help it. I've loved him since Armageddon #eonlinechat")
# ct = tf.constant(ct)
# to_feature(ct, tf.constant(0))

# Task 7: Wrap a Python Function into a TensorFlow op for Eager Execution

In [ ]:
def to_feature_map(text, label):
  out = tf.py_function(to_feature, inp=[text, label], 
                       Tout=[tf.int32, tf.int32, tf.int32, tf.int32,])
  iids, imask, segids, label = out[0], out[1], out[2], out[3]

  iids.set_shape([max_seq_length])
  imask.set_shape([max_seq_length])
  segids.set_shape([max_seq_length])
  label.set_shape([])

  x = {
        "input_word_ids": iids,
        "input_mask": imask,
        "input_type_ids": segids
  }
  return (x, label)

In [ ]:
#testing the data
s = train_data.take(1)
for t, l in s:
  # print(t)
  inps = to_feature_map(t, l)[0]  
  print(inps)

In [ ]:
with tf.device('/cpu:0'):
  # train
  train_data = (train_data
                .map(to_feature_map, num_parallel_calls=tf.data.AUTOTUNE)
                .shuffle(1000)
                .batch(train_batch_size, drop_remainder=True)
                .prefetch(tf.data.AUTOTUNE))

  # valid
  
  test_data = (test_data
                .map(to_feature_map, num_parallel_calls=tf.data.AUTOTUNE)
                # .shuffle(1000) we dont need to shuffle the test data, just need them for predictions
                .batch(train_batch_size, drop_remainder=True)
                .prefetch(tf.data.AUTOTUNE))

In [ ]:
# train data spec
train_data.element_spec

# Add a Classification Head to the BERT Layer

In [ ]:
from tensorflow.keras import layers
# Train model
from datetime import datetime
from tensorflow.keras import callbacks

In [ ]:
# Building the model
def create_model(drop_rate=0.4, m_name="model"):
  input_word_ids = layers.Input(shape=(max_seq_length, ),
                             dtype=tf.int32,
                             name="input_word_ids")
  input_mask = layers.Input(shape=(max_seq_length, ),
                             dtype=tf.int32,
                             name="input_mask")
  input_type_ids = layers.Input(shape=(max_seq_length, ),
                             dtype=tf.int32,
                             name="input_type_ids")
  # pooled output is vector representation of the whole sentence
  # sequesnced output is vector representation of each word
  inp_as_dict = {
      "input_word_ids": input_word_ids,
      "input_mask": input_mask,
      "input_type_ids": input_type_ids
  }
  inp_as_lst = [input_word_ids, input_mask, input_type_ids]

  bert_outputs = bert_layer(inp_as_dict)
  pooled_output = bert_outputs["pooled_output"]      # [batch_size, 768].
  sequence_output = bert_outputs["sequence_output"]  # [batch_size, seq_length, 768].


  # x = layers.Dense(512, activation = 'relu')(pooled_output)
  # x = layers.Dropout(0.5)(x)
  # x = layers.Dense(256, activation = 'relu')(x)
  # x = layers.Dropout(0.2)(x)
  # x = layers.Dense(64, activation = 'relu')(x)
  x = layers.Dropout(drop_rate)(pooled_output)
  outputs = layers.Dense(1, activation="sigmoid", name="outputs")(x)

  model = tf.keras.Model(
      #mapping the input dict values here
      inputs = {
            'input_word_ids': input_word_ids,
            'input_mask': input_mask,
            'input_type_ids': input_type_ids,
      }, 
      outputs=outputs, name=m_name)
  
  return model

In [ ]:
def compile_model(model, lr=2e-5):
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                loss=tf.keras.losses.BinaryCrossentropy(),
                metrics=["accuracy"])

  model_fig = tf.keras.utils.plot_model(model, show_shapes=True, dpi=72)
  return model, model_fig

In [ ]:
model = create_model(m_name="model0")
model.summary()

In [ ]:
model, _ = compile_model(model, 1e-6)

In [ ]:
!nvidia-smi

In [ ]:
now = datetime.now()
curr_time = now.strftime("%Y%m%d_%H%M%S")
filepath = "01Disaster_tweet/dense" + curr_time  
chk_pt = callbacks.ModelCheckpoint(filepath=filepath,
                                    save_weights_only=True,
                                    verbose=0)

EPOCHS = 6
history = model.fit(train_data,
                    validation_data=test_data,
                    epochs=EPOCHS,
                    callbacks=[chk_pt])

In [ ]:
import matplotlib.pyplot as plt

def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])
  plt.show()

In [ ]:
plot_graphs(history, "accuracy")

In [ ]:
TEST_URL = "https://raw.githubusercontent.com/gkv856/KaggleData/main/test.csv"
df_test = pd.read_csv(TEST_URL)
df_test.head()

In [ ]:

# num = 5
# test_sample = df_test["text"].head(num)
# ids = df_test['id'].head(num)
# test_sample

num = 5
test_sample = df_test["text"]#.head(num)
ids = df_test['id']#.head(num)
test_sample

In [ ]:
# making preditions on the real test set
real_test_data = tf.data.Dataset.from_tensor_slices((test_sample, [0]*len(test_sample)))
real_test_data = (real_test_data.map(to_feature_map).batch(train_batch_size)
                  .prefetch(tf.data.AUTOTUNE))

y_preds = model.predict(real_test_data) 
y_preds = tf.round(y_preds)
y_preds = tf.cast(y_preds, dtype=tf.int32)

res = tf.squeeze(y_preds).numpy()
my_sub = pd.DataFrame({'id': ids,'target':res})

my_sub.head(20)

In [ ]:
my_sub.to_csv("sub_clean_data_slow_lr.csv", index=False)

In [ ]:

EPOCHS = 6
history = model.fit(train_data,
                    validation_data=test_data,
                    epochs=EPOCHS,
                    callbacks=[chk_pt])

In [ ]:
# making preditions on the real test set
real_test_data = tf.data.Dataset.from_tensor_slices((test_sample, [0]*len(test_sample)))
real_test_data = (real_test_data.map(to_feature_map).batch(train_batch_size)
                  .prefetch(tf.data.AUTOTUNE))

y_preds = model.predict(real_test_data) 
y_preds = tf.round(y_preds)
y_preds = tf.cast(y_preds, dtype=tf.int32)

res = tf.squeeze(y_preds).numpy()
my_sub = pd.DataFrame({'id': ids,'target':res})

my_sub.head(20)

In [ ]:
my_sub.to_csv("submission.csv", index=False)